# 달러1달리치

* 달러리치 앱 - 달러투자 가이드
  - https://m.blog.naver.com/PostView.naver?blogId=boot&logNo=222430430028&proxyReferer=

* 야후 finance 모듈 설치

In [1]:
!pip install yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 6.3 MB 8.6 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=9314b2fdf327d8bc18ba8328552f8266ecf297092758c0828a5821b518adb4d0
  Stored in directory: /tmp/pip-ephem-wheel-cache-4p8rgbi8/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


* 야후 finance 모듈 import

In [2]:
import yfinance as yf

In [3]:
obj = yf.Ticker("KRW=X")
obj.info

{'algorithm': None,
 'ask': 1182.52,
 'askSize': 0,
 'averageDailyVolume10Day': 0,
 'averageVolume': 0,
 'averageVolume10days': 0,
 'beta': None,
 'bid': 1181.52,
 'bidSize': 0,
 'circulatingSupply': None,
 'currency': 'KRW',
 'dayHigh': 1181.52,
 'dayLow': 1181.52,
 'dividendRate': None,
 'dividendYield': None,
 'exDividendDate': None,
 'exchange': 'CCY',
 'exchangeTimezoneName': 'Europe/London',
 'exchangeTimezoneShortName': 'BST',
 'expireDate': None,
 'fiftyDayAverage': 1160.1884,
 'fiftyTwoWeekHigh': 1183.5,
 'fiftyTwoWeekLow': 1079.22,
 'fiveYearAvgDividendYield': None,
 'forwardPE': None,
 'fromCurrency': None,
 'gmtOffSetMilliseconds': '3600000',
 'isEsgPopulated': False,
 'lastMarket': None,
 'logo_url': '',
 'market': 'ccy_market',
 'marketCap': None,
 'maxAge': 1,
 'maxSupply': None,
 'messageBoardId': 'finmb_KRW_X',
 'navPrice': None,
 'open': 1181.52,
 'openInterest': None,
 'payoutRatio': None,
 'previousClose': 1175.2,
 'priceHint': 4,
 'priceToSalesTrailing12Months': No

In [4]:
obj = yf.Ticker("DX-Y.NYB")
obj.info

{'algorithm': None,
 'ask': 0,
 'askSize': 0,
 'averageDailyVolume10Day': 0,
 'averageVolume': 0,
 'averageVolume10days': 0,
 'beta': None,
 'bid': 0,
 'bidSize': 0,
 'circulatingSupply': None,
 'currency': 'USD',
 'dayHigh': 93.248,
 'dayLow': 92.765,
 'dividendRate': None,
 'dividendYield': None,
 'exDividendDate': None,
 'exchange': 'NYB',
 'exchangeTimezoneName': 'America/New_York',
 'exchangeTimezoneShortName': 'EDT',
 'expireDate': None,
 'fiftyDayAverage': 92.687775,
 'fiftyTwoWeekHigh': 94.74,
 'fiftyTwoWeekLow': 89.21,
 'fiveYearAvgDividendYield': None,
 'forwardPE': None,
 'fromCurrency': None,
 'gmtOffSetMilliseconds': '-14400000',
 'isEsgPopulated': False,
 'lastMarket': None,
 'logo_url': '',
 'market': 'us24_market',
 'marketCap': None,
 'maxAge': 1,
 'maxSupply': None,
 'messageBoardId': 'finmb_INDEXDX-Y.NYB',
 'navPrice': None,
 'open': 92.876,
 'openInterest': None,
 'payoutRatio': None,
 'previousClose': 93.195,
 'priceHint': 2,
 'priceToSalesTrailing12Months': None,


* get_ticker_hist(symbol, period) : period 기간의 symbol에 대해 시고저종 데이터프레임을 얻는다.
  - symbol 
    - KRW=X : 원/달러
    - DX-Y.NYB : 달러인덱스
  - period : 1mo(1달), 1y(1년)

In [5]:
import datetime

def get_ticker_hist(symbol, period):
    obj = yf.Ticker(symbol)
    hist_df = obj.history(period=period, auto_adjust=False)
    if abs(hist_df['Close'].iloc[-1] - obj.info['regularMarketPrice']) > 0.1:
        hist_df = hist_df.reset_index()
        new_row = {
          'Date' : datetime.datetime.now(),
          'Open' : obj.info['regularMarketOpen'],
          'High' : obj.info['regularMarketDayHigh'],
          'Low' : obj.info['regularMarketDayLow'],
          'Close' : obj.info['regularMarketPrice'],
          'Adj Close' : obj.info['regularMarketPrice'],
          'Volume' : 0,
          'Dividends' : 0,
          'Volume' : 0
        }
        hist_df = hist_df.append(new_row,ignore_index=True)
        hist_df = hist_df.set_index('Date')
    return hist_df

def get_ticker_values(hist):
    low = hist['Low'].min()
    high = hist['High'].max()
    mid = (high + low) / 2.0
    close = hist['Close'].iloc[-1]
    values_dic = {
      "low": low,      
      "high": high,      
      "mid" : mid,
      "close" : close,
      "is_buy" : close < mid
    }
    return values_dic

## 1. 원달러 환율

In [6]:
# 원달러 환율 1년 데이터프레임 얻기
#hist = get_ticker_hist("KRW=X", "1mo")
hist = get_ticker_hist("KRW=X", "1y")

In [7]:
hist

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
2020-09-17 00:00:00.000000,1173.800049,1177.229980,1169.780029,1172.069946,1172.069946,0,0,0.0
2020-09-18 00:00:00.000000,1168.760010,1168.760010,1158.640015,1168.650024,1168.650024,0,0,0.0
2020-09-21 00:00:00.000000,1163.979980,1167.270020,1155.589966,1163.989990,1163.989990,0,0,0.0
2020-09-22 00:00:00.000000,1162.099976,1166.000000,1158.400024,1163.849976,1163.849976,0,0,0.0
2020-09-23 00:00:00.000000,1160.800049,1169.329956,1160.099976,1162.500000,1162.500000,0,0,0.0
...,...,...,...,...,...,...,...,...
2021-09-14 00:00:00.000000,1171.640015,1174.199951,1165.040039,1171.109985,1171.109985,0,0,0.0
2021-09-15 00:00:00.000000,1170.750000,1173.859985,1165.160034,1170.800049,1170.800049,0,0,0.0
2021-09-16 00:00:00.000000,1165.099976,1175.829956,1165.099976,1165.219971,1165.219971,0,0,0.0


* low : 기간(1년, 1달) 최저가
* high : 기간(1년, 1달) 최고가
* mid : low와 high의 중간가
* close : 현재가
* is_buy : 현재가가 기간(1년, 1달) 중간가보다 작으면 True, 아니면 False

In [8]:
usdkrw = get_ticker_values(hist)
usdkrw

{'close': 1181.52,
 'high': 1183.5,
 'is_buy': False,
 'low': 1079.219970703125,
 'mid': 1131.3599853515625}

## 2. 달러 인덱스 (달러 지수)

In [9]:
# 달러인덱스 1년 데이터프레임 얻기
#hist = get_ticker_hist("DX-Y.NYB", "1mo")
hist = get_ticker_hist("DX-Y.NYB", "1y")
hist

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
2020-09-18,92.860001,93.010002,92.769997,92.930000,92.930000,0,0,0
2020-09-21,93.010002,93.779999,92.750000,93.540001,93.540001,0,0,0
2020-09-22,93.550003,94.080002,93.470001,93.989998,93.989998,0,0,0
2020-09-23,93.989998,94.430000,93.930000,94.389999,94.389999,0,0,0
2020-09-24,94.379997,94.589996,94.199997,94.349998,94.349998,0,0,0
...,...,...,...,...,...,...,...,...
2021-09-13,92.610001,92.889999,92.570000,92.620003,92.620003,0,0,0
2021-09-14,92.620003,92.680000,92.320000,92.650002,92.650002,0,0,0
2021-09-15,92.680000,92.680000,92.419998,92.489998,92.489998,0,0,0


* low : 기간(1년, 1달) 최저가
* high : 기간(1년, 1달) 최고가
* mid : low와 high의 중간가
* close : 현재가
* is_buy : 현재가가 기간(1년, 1달) 중간가보다 작으면 True, 아니면 False

In [10]:
dxy = get_ticker_values(hist)
dxy

{'close': 93.19999694824219,
 'high': 94.73999786376953,
 'is_buy': False,
 'low': 89.20999908447266,
 'mid': 91.9749984741211}

## 3. 달러 갭 비율
* 산출 공식 : 달러 지수 / (원/달러 환율) * 100 = 달러 갭 비율
* mid : 달러 지수 중간값 / (원/달러 환율 중간값) * 100
* close : 달러 지수 현재가 / (원/달러 환율 현재가) * 100
* is_buy : 현재가가 기간(1년, 1달) 중간값보다 크면 True, 아니면 False

In [11]:
def get_gap(usdkrw, dxy):
    gap_dic = {}
    mid = dxy.get("mid", 0) / usdkrw.get("mid", 0) * 100.0
    close = dxy.get("close", 0) / usdkrw.get("close", 0) * 100.0
    if dxy.get("mid", 0) and usdkrw.get("mid", 0):
        gap_dic["mid"] = mid
    if dxy.get("close", 0) and usdkrw.get("close", 0):
        gap_dic["close"] = close
    gap_dic["is_buy"] = close > mid
    return gap_dic

In [12]:
gap_rate = get_gap(usdkrw, dxy)
gap_rate

{'close': 7.888143827293841, 'is_buy': False, 'mid': 8.129596208543692}

## 4. 적정 환율
* 산출 공식 : 현재의 달러 지수 / 52주 중간 달러 갭 비율 * 100 = 적정 환율

In [13]:
def get_proper_usdkrw(dxy, gap_rate):
    value = 0
    if dxy.get("close", 0) and gap_rate.get("mid", 0):
        value = dxy.get("close", 0) / gap_rate.get("mid", 0) * 100.0
    return value

In [14]:
get_proper_usdkrw(dxy, gap_rate)

1146.428365658492